In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as web
import datetime as dt
import pandas_ta as ta
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn import preprocessing


In [ ]:
data = pd.read_csv('../Datasets/bitfinex_ETHUSDT1h.csv')
data['date'] = pd.to_datetime(data['date'], unit='ms')
date.head()

In [ ]:
data['date'] = pd.to_datetime(data['date'],utc=True)
data.date

In [38]:
array = [[1,'W'],[1,'D'],[12,'h'],[6,'h'],[4,'h'],[3,'h'],[1,'h'],[30,'m'],[15,'m'],[5,'m'],[1,'m']]

for number, unit in array:
   


    data = pd.read_csv( 'ETHUSDT-'+str(number) + unit + '-bitfinex.csv')
    data['date'] = pd.to_datetime(data['date'],utc=True)
    saeid = convert_date(data[['date','close']])
    saeid.to_csv('../Datasets/converted_date_ETHUSDT-'+str(number) + unit + '-bitfinex.csv')

/tmp/ipykernel_709495/420165420.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["day"] = temp_df.date.dt.day
/tmp/ipykernel_709495/420165420.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["month"] = temp_df.date.dt.month
/tmp/ipykernel_709495/420165420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [ ]:
def convert_date(temp_df: pd.DataFrame):
    
   
    temp_df["day"] = temp_df.date.dt.day
    temp_df["month"] = temp_df.date.dt.month
    temp_df["year"] = temp_df.date.dt.year
    temp_df["hour"] = temp_df.date.dt.hour
    temp_df["day_of_week"] = temp_df.date.dt.dayofweek
    temp_df["season"] = temp_df["month"].apply(lambda x: int((((x + 1) % 12) - (((x + 1) % 12) % 3)) / 3))
    encoder = OneHotEncoder()
    season_one_hot_encode = encoder.fit_transform(temp_df["season"].to_numpy().reshape(-1, 1)).toarray()
    seasons_df = pd.DataFrame(season_one_hot_encode, columns=["winter", "spring", "summer", "fall"])
    temp_df = temp_df.join(seasons_df)
    temp_df = temp_df.drop(columns=["season"])
    selected_columns = ["day", "month", "hour", "day_of_week"]
    for column in selected_columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        temp_df[column] = scaler.fit_transform(temp_df[column].to_numpy().reshape(-1, 1)) * np.pi
        temp_df[column + "_x"] = np.sin(temp_df[column])
        temp_df[column + "_y"] = np.cos(temp_df[column])

    # temp_df = temp_df.drop(columns=selected_columns + ["date"])
    temp_df = temp_df.drop(columns=selected_columns)

    return temp_df



def add_basic_macds(data : pd.DataFrame):
    
    data['hl2'] = (ta.hl2(high=data['high'], low=data['low'])).values
    data['hlc3'] = (ta.hlc3(high=data['high'], low=data['low'], close=data['close'])).values
    data['ohlc4'] = (ta.ohlc4(high=data['high'], low=data['low'], close=data['close'],open_=data['open'])).values

    korosh = ta.macd(data['hl2'])
    data['hl2_MD'] = korosh['MACD_12_26_9']
    data['hl2_MDh'] = korosh['MACDh_12_26_9']
    data['hl2_MDs'] = korosh['MACDs_12_26_9']


    korosh = ta.macd(data['hlc3'])
    data['hlc3_MD'] = korosh['MACD_12_26_9']
    data['hlc3_MDh'] = korosh['MACDh_12_26_9']
    data['hlc3_MDs'] = korosh['MACDs_12_26_9']



    korosh = ta.macd(data['ohlc4'])
    data['ohlc4_MD'] = korosh['MACD_12_26_9']
    data['ohlc4_MDh'] = korosh['MACDh_12_26_9']
    data['ohlc4_MDs'] = korosh['MACDs_12_26_9']
    return data


def add_all_macds_by_arman( data : pd.DataFrame ):
    import numpy as np
    import itertools
    import pandas as pd
    import pandas_ta as ta
    import time
    from tqdm import tqdm
    from functools import partial
    from dateutil import parser
    from multiprocessing import Pool
    import multiprocessing
    from multiprocessing import Process, Lock

    from pandas import concat, DataFrame
    from pandas_ta import wma
    from pandas_ta.overlap import ema, sma
    from pandas_ta.utils import get_offset, verify_series, signals
    
    
    

    def macd_indicator(close, fast=None, slow=None, signal=None, talib=None, offset=None, matype=None, **kwargs):
        """Indicator: Moving Average, Convergence/Divergence (MACD)"""
        # Validate arguments
        fast = int(fast) if fast and fast > 0 else 12
        slow = int(slow) if slow and slow > 0 else 26
        signal = int(signal) if signal and signal > 0 else 9
        if slow < fast:
            fast, slow = slow, fast
        close = verify_series(close, max(fast, slow, signal))
        offset = get_offset(offset)
        mode_tal = bool(talib) if isinstance(talib, bool) else True

        if close is None: return

        as_mode = kwargs.setdefault("asmode", False)

        # Calculate Result

        if matype == "ema" or matype is None:
            fastma = ema(close, length=fast)
            slowma = ema(close, length=slow)

            macd = fastma - slowma
            signalma = ema(close=macd.loc[macd.first_valid_index():, ], length=signal)
            histogram = macd - signalma
        elif matype == "sma":
            fastma = sma(close, length=fast)
            slowma = sma(close, length=slow)

            macd = fastma - slowma
            signalma = sma(close=macd.loc[macd.first_valid_index():, ], length=signal)
            histogram = macd - signalma
        elif matype == "wma":
            fastma = wma(close, length=fast)
            slowma = wma(close, length=slow)

            macd = fastma - slowma
            signalma = wma(close=macd.loc[macd.first_valid_index():, ], length=signal)
            histogram = macd - signalma

        if as_mode:
            if matype == "ema" or matype is None:
                macd = macd - signalma
                signalma = ema(close=macd.loc[macd.first_valid_index():, ], length=signal)
                histogram = macd - signalma
            elif matype == "sma":
                macd = macd - signalma
                signalma = sma(close=macd.loc[macd.first_valid_index():, ], length=signal)
                histogram = macd - signalma
            elif matype == "wma":
                macd = macd - signalma
                signalma = wma(close=macd.loc[macd.first_valid_index():, ], length=signal)
                histogram = macd - signalma

        # Offset
        if offset != 0:
            macd = macd.shift(offset)
            histogram = histogram.shift(offset)
            signalma = signalma.shift(offset)

        # Handle fills
        if "fillna" in kwargs:
            macd.fillna(kwargs["fillna"], inplace=True)
            histogram.fillna(kwargs["fillna"], inplace=True)
            signalma.fillna(kwargs["fillna"], inplace=True)
        if "fill_method" in kwargs:
            macd.fillna(method=kwargs["fill_method"], inplace=True)
            histogram.fillna(method=kwargs["fill_method"], inplace=True)
            signalma.fillna(method=kwargs["fill_method"], inplace=True)

        # Name and Categorize it
        _asmode = "AS" if as_mode else ""
        _props = f"_{fast}_{slow}_{signal}"
        macd.name = f"MACD{_asmode}{_props}"
        histogram.name = f"MACD{_asmode}h{_props}"
        signalma.name = f"MACD{_asmode}s{_props}"
        macd.category = histogram.category = signalma.category = "momentum"

        # Prepare DataFrame to return
        data = {macd.name: macd, histogram.name: histogram, signalma.name: signalma}
        df = DataFrame(data)
        df.name = f"MACD{_asmode}{_props}"
        df.category = macd.category

        signal_indicators = kwargs.pop("signal_indicators", False)
        if signal_indicators:
            signalsdf = concat(
                [
                    df,
                    signals(
                        indicator=histogram,
                        xa=kwargs.pop("xa", 0),
                        xb=kwargs.pop("xb", None),
                        xserie=kwargs.pop("xserie", None),
                        xserie_a=kwargs.pop("xserie_a", None),
                        xserie_b=kwargs.pop("xserie_b", None),
                        cross_values=kwargs.pop("cross_values", True),
                        cross_series=kwargs.pop("cross_series", True),
                        offset=offset,
                    ),
                    signals(
                        indicator=macd,
                        xa=kwargs.pop("xa", 0),
                        xb=kwargs.pop("xb", None),
                        xserie=kwargs.pop("xserie", None),
                        xserie_a=kwargs.pop("xserie_a", None),
                        xserie_b=kwargs.pop("xserie_b", None),
                        cross_values=kwargs.pop("cross_values", False),
                        cross_series=kwargs.pop("cross_series", True),
                        offset=offset,
                    ),
                ],
                axis=1,
            )

            return signalsdf
        else:
            return df

        
    def get_indicator_elements(indicator: str):
        if indicator == "rsi":
            rsi_range = [['period', 5, 25],
                         ['overbuy', 60, 100],
                         ['oversell', 0, 40],
                         ['source', 'low', 'high', 'close', 'open', 'hlc2', 'hlc3', 'ohlc4']
                         ]
            return get_elements(rsi_range, setting_size=4)
        if indicator == "macd":
            macd_range = [['fast', 5, 12],
                          ['slow', 11, 20],
                          ['signal', 22, 48],
                          ['source', 'low', 'high', 'close', 'open', 'hlc2', 'hlc3', 'ohlc4'],
                          ['MA', 'ema', 'sma', 'wma']
                          ]
            return get_elements(macd_range, setting_size=5)
        if indicator == "stochrsi":
            stochrsi_range = [['k', 3, 5],
                              ['d', 3, 5],
                              ['length', 50, 57],
                              ['rsi_length', 50, 57],
                              ['source', 'low', 'high', 'close', 'open', 'hlc2', 'hlc3', 'ohlc4'],
                              ['overbuy', 70, 100],
                              ['oversell', 0, 30]
                              ]
            return get_elements(stochrsi_range, setting_size=7)






    def get_source(data: pd.DataFrame, source: str = 'close'):


        saeid = {
            'hl2': data.ta.hl2(),
            'hlc3': data.ta.hlc3(),
            'ohlc4': data.ta.ohlc4(),
            'close': data['close'],
            'high': data['high'],
            'low': data['low'],
            'open': data['open']
        }.get(source, data['close'])

        return saeid


    def get_elements(test, setting_size: int):
        adad = []
        for i in range(len(test)):
            if isinstance(test[i][1], int):
                adad.append(np.arange(test[i][1], test[i][2] + 1, 1).tolist())
            elif isinstance(test[i][1], str):
                adad.append((test[i])[1:])

      
        if setting_size == 4:
            elements = []
            for element in itertools.product(adad[0], adad[1], adad[2], adad[3]):
                elements.append(list(element))
            return elements
        if setting_size == 5:
            elements = []
            for element in itertools.product(adad[0], adad[1], adad[2], adad[3], adad[4]):
                elements.append(list(element))
            return elements
        if setting_size == 7:
            elements = []
            for element in itertools.product(adad[0], adad[1], adad[2], adad[3], adad[4], adad[5], adad[6]):
                elements.append(list(element))
            return elements

    def correlation(dataset, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        return col_corr
    def create_macds(obj , dataset):
        macd = macd_indicator(close=get_source(data = data, source=obj[3]), fast=obj[0],
                          slow=obj[1],
                          signal=obj[2],
                          matype=obj[4])

        macd = macd.drop(macd.columns[1] , axis=1)
        macd.columns = [macd.columns[0]+""+obj[3]+""+obj[4] , macd.columns[1]+""+obj[3]+""+obj[4]]
        return macd


    from tqdm.contrib.concurrent import thread_map , process_map
    from tqdm import tqdm

    remove = ['open','high','low','close','volume']
    elements = get_indicator_elements('macd')

    feature_data = data.copy(deep=True)
    feature_data = feature_data.drop(['date'] , axis =1)

    step = 300
    for i in tqdm(range(0,len(elements),step)):
        if (i+step) >= len(elements) :

            sub_elements = elements[i:]
        else:

            sub_elements = elements[i:i+step]


        pool = Pool(12)

        r_macd = partial(create_macds, dataset=data.copy(deep=True))
        start_time = time.time()
        f = process_map(r_macd, sub_elements)
        df = pd.concat(f , axis=1)
        print(np.shape(feature_data))

        for i in(df.columns):

            if (df[i] == 0).any():
                if (df[i].value_counts())[0] > 1000:
                    df = df.drop(i,axis=1)





        frames = [feature_data, df]
        feature_data = pd.concat(frames , axis = 1 )






        corr_features = correlation(feature_data, 0.85)


        for i in remove:

            if i in corr_features:
                corr_features.remove(i)

        feature_data = feature_data.drop(corr_features,axis=1)
    return feature_data




    

In [ ]:
"""
Mr.Kataei 8/4/2021
get_candle_api use in stream to get candles for analysis and strategies this method have limit for n last candles
you need for analysis.data collect form Bitfinex API where url can change for any API you want.
there is 3 type of dictionary for bitfinex symbols or CSvs we need to save
_get_all_candles is private method to use in generate_big_data where collect all data that can get from bitfinex
"""
import time
import pandas as pd
import requests
import enum


class Exchange(enum.Enum):
    binance = 'binance'
    bitfinex = 'bitfinex'


class Symbols(enum.Enum):
    BTCUSDT = 'bitcoin'
    ETHUSDT = 'ethereum'
    ADAUSDT = 'cardano'
    DOGEUSDT = 'doge'
    BCHUSDT = 'bitcoinCash'
    ETCUSDT = 'ethereumClassic'


symbols = {'BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOGEUSDT', 'BCHUSDT', 'ETCUSDT'}

symbols_bitfinix = {'BTCUSDT': 'tBTCUSD', 'ETHUSDT': 'tETHUSD', 'ADAUSDT': 'tADAUSD', 'DOGEUSDT': 'tDOGE:USD',
                    'BCHUSDT': 'tBCHN:USD', 'ETCUSDT': 'tETCUSD'}


def get_url_params_column(exchange: Exchange, timeframe: str, symbol: str, start_time: int = 115133520000):
    # 115133520000 is Wednesday, 21 December 2005 02:52:00
    if exchange.name == Exchange.binance.name:
        params = {'interval': timeframe, 'symbol': symbol, 'limit': 1000, 'startTime': start_time}
        url = 'https://api1.binance.com/api/v3/klines'
        columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'Qav', 'trade-number', 'TBbav',
                   'TBbqv', 'ignore']
    elif exchange.name == Exchange.bitfinex.name:
        symbol = symbols_bitfinix[symbol]
        params = {'limit': 10000, 'sort': 1, 'start': start_time}
        url = f'https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:{symbol}/hist'
        columns = ['date', 'open', 'close', 'high', 'low', 'volume']
    else:
        return
    return params, url, columns


def calculate_end_time(number: int = 4, unit: str = 'h'):
    convert_time_to_ms = 0
    if unit == 'h':
        convert_time_to_ms = 3600 * 1000
    elif unit == 'm':
        convert_time_to_ms = 60 * 1000
    elif unit == 'd':
        convert_time_to_ms = 3600 * 1000 * 24

    end_time = int(round(time.time() * 1000))
    return end_time - number * convert_time_to_ms


def get_all_candles_binance(exchange: Exchange, symbol: Symbols, number: int = 4, unit: str = 'h',
                            is_tehran: bool = True, save_csv: bool = True):
    """
    :param exchange: its enum from Exchange ( bitfinex , binance ,...)
    :param symbol: its enum from Symbols
    :param number: its int number number of day or hours or minutes
    :param unit: its string for which timeframe,  m for minutes , h for hours , d for day
    :param is_tehran: for casting to tehran time this parameter need to be true
    :param save_csv:  for save data to directory calls this function needs to be true
    :return: dataframe of ohcl
    """
    timeframe = str(number) + unit
    symbol = symbol.name
    params, url, columns = get_url_params_column(exchange=exchange, timeframe=timeframe, symbol=symbol)
    if params is None:
        return

    end_time = calculate_end_time(number=number, unit=unit)

    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=columns).astype(float)
        data = data[data.columns[:]]
        start_time = int(data.tail(1)['date'].values[0])
        while start_time < end_time:
            params, url, columns = get_url_params_column(exchange=exchange, timeframe=timeframe, symbol=symbol,
                                                         start_time=start_time)
            r = requests.get(url=url, params=params)
            next_data = r.json()
            next_data = pd.DataFrame(data=next_data, columns=columns).astype(float)
            next_data = next_data[data.columns[:]]
            data = pd.concat([data, next_data], axis=0)
            data = data.reset_index(drop=True)
            start_time = int(data.tail(1)['date'].values[0])
        if is_tehran:
            data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)).tz_localize(
                'UTC').tz_convert('Asia/Tehran')
        data = data.drop_duplicates(subset=['date'])
        if save_csv:
            data.to_csv(f'{symbol}-{timeframe}-{exchange.name}.csv', index=False)
        return data
    except Exception as e:
        print(f'something wrong on get data from {exchange.name}:\n', e)


def get_candle_bitfinex(symbol: str, timeframe: str, limit: int):
    symbol = symbols_bitfinix[symbol]
    params = {'limit': limit, 'sort': -1}
    url = f'https://api-pub.bitfinex.com/v2/candles/trade:{timeframe}:{symbol}/hist'
    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=['date', 'open', 'close', 'high', 'low', 'volume'])
        data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)
                                     ).tz_localize('UTC').tz_convert('Asia/Tehran')
        data = data.iloc[::-1]
        data = data.reset_index(drop=True)
        return data
    except Exception as e:
        print('something wrong on get data from bitfinex:\n', e)


def get_candle_binance(symbol: str, limit: int, number: int = 4, unit: str = 'h'):
    timeframe = str(number) + unit
    end_time = calculate_end_time(number=number, unit=unit)
    params = {'interval': timeframe, 'symbol': symbol, 'limit': limit, 'endTime': end_time}
    url = 'https://api1.binance.com/api/v3/klines'
    try:
        r = requests.get(url=url, params=params)
        data = r.json()
        data = pd.DataFrame(data=data, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                                'Qav', 'trade-number', 'TBbav', 'TBbqv', 'ignore']).astype(float)
        data = data[data.columns[0:6]]
        data.date = pd.DatetimeIndex(pd.to_datetime(data['date'], unit='ms', yearfirst=True)).tz_localize(
            'UTC').tz_convert(
            'Asia/Tehran')
        return True, data
    except Exception as e:
        result = 'something wrong getting data from binance:\n' + str(e)
        return False, result




In [ ]:
"""
    Mr.kataei 1/3/2022
"""
from Libraries.data_collector import *
import pandas_ta as ta
import pandas as pd

exchange = Exchange.bitfinex
symbol = Symbols.BTCUSDT



def willr(dataframe: pd.DataFrame, length: int = 14):
    try:
        return ta.willr(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'], length=length)
    except Exception as e:
        print(f'invalid dataframe: {e}')


def cmo(dataframe: pd.DataFrame, length: int = 14, scalar: int = 100):
    try:
        return ta.cmo(close=dataframe['close'], length=length, scalar=scalar)
    except Exception as e:
        print(f'invalid dataframe: {e}')


def mfi(dataframe: pd.DataFrame, length: int = 14):
    try:
        return ta.mfi(high=dataframe['high'], low=dataframe['low'], close=dataframe['close'],
                      volume=dataframe['volume'], length=length)
    except Exception as e:
        print(f'invalid dataframe: {e}')


def stochRsi(dataframe: pd.DataFrame, length: int = 14, rsi_length: int = 14, k: int = 3, d: int = 3):
    try:
        return ta.stochrsi(close=dataframe['close'], length=length, rsi_length=rsi_length, k=k, d=d)
    except Exception as e:
        print(f'invalid dataframe: {e}')


def generate_dataset(dataframe: pd.DataFrame):
    """
    **need must functional in future**
    :param dataframe: dataframe ohclv
    :return: indicators with dataframe
    """
    stoch_rsi_indicator = stochRsi(dataframe=dataframe)
    mfi_indicator = mfi(dataframe=dataframe)
    cmo_indicator = cmo(dataframe=dataframe)
    willr_indicator = willr(dataframe=dataframe)

    temp = pd.concat([stoch_rsi_indicator, mfi_indicator, cmo_indicator, willr_indicator], axis=1)
    temp.columns = ["stoch_k", "stoch_d", "mfi", "cmo", "willr"]
    dataframe = pd.concat([dataframe, temp], axis=1)
    return 

In [ ]:
saeid = convert_date(data[['date','close']])

In [ ]:
saeid.head()

In [ ]:
saeid.to_csv('../converted_date_all_4h_ETHUSDT.csv')

In [ ]:
add_all_macds_by_arman(pd.read_csv('../ETHUSDT4h.csv'))